In [359]:
import numpy as np
import collections
import os
from abc import ABC, abstractmethod
from typing import Type
from numpy.linalg import norm

#########################################

import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable

In [360]:
show_logs = False

In [361]:
# HELPER FUNCTIONS
def _read_words(filename):
    with open(filename, "r") as f:
      return ["<bos>"] + f.read().replace("\n", "<eos> <bos>").split()[:-1]

def _build_vocab(filename):
    data = _read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    id_to_word = dict((v, k) for k, v in word_to_id.items())

    return word_to_id, id_to_word

def _file_to_word_ids(filename, word_to_id):
    data = _read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

# Processes the raw data from text files
def ptb_raw_data(data_path=None, prefix="ptb"):
    train_path = os.path.join(data_path, prefix + ".train.txt")
    valid_path = os.path.join(data_path, prefix + ".valid.txt")
    test_path = os.path.join(data_path, prefix + ".test.txt")

    word_to_id, id_2_word = _build_vocab(train_path)
    train_data = _file_to_word_ids(train_path, word_to_id)
    valid_data = _file_to_word_ids(valid_path, word_to_id)
    test_data = _file_to_word_ids(test_path, word_to_id)
    return train_data, valid_data, test_data, word_to_id, id_2_word

# Yields minibatches of data
def ptb_iterator(raw_data, batch_size, num_steps):
    raw_data = np.array(raw_data, dtype=np.int32)

    data_len = len(raw_data)
    batch_len = data_len // batch_size
    print(f'training data shape is {data_len} and the number of batches is {batch_len}')
    data = np.zeros([batch_size, batch_len], dtype=np.int32)
    for i in range(batch_size):
        data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

    epoch_size = (batch_len - 1) // num_steps

    if epoch_size == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

    for i in range(epoch_size):
        x = data[:, i*num_steps:(i+1)*num_steps]
        y = data[:, i*num_steps+1:(i+1)*num_steps+1]
        yield (x, y)

In [703]:
# From previous project
class Param(ABC):
    def __init__(self, is_trainable : bool = True, initialization_method : str = "random"):
        self.is_trainable = is_trainable
        self.initialization_method = initialization_method
        self.param = None
        self.dparam = None
        self.dparam_1 = None

    def __str__(self,):
        return self.param

    @abstractmethod
    def initialize_parameters(self, ):
        pass

class ScalarParam(Param):
    def __init__(self, trainable : bool = True, initialization_method : str = "normal"):
        super().__init__(trainable, initialization_method)
        self.initialize_parameters()

    def initialize_parameters(self,):
        if self.initialization_method=="normal":
            self.param = np.random.random()
        elif self.initialization_method=="zero":
            self.param = 0.0
        elif self.initialization_method=="one":
            self.param = 1.0

        self.dparam = 0
        self.dparam_1 = None

class ArrayParam(Param):
    def __init__(self, shape: tuple, is_trainable : bool = True, initialization_method : str = "normal", **kwargs):
        super().__init__(is_trainable, initialization_method)
        self.shape = shape
        self.loc = kwargs['loc'] if 'loc' in kwargs else 0.
        self.scale = kwargs['scale'] if 'scale' in kwargs else 1.
        self.low = kwargs['low'] if 'low' in kwargs else -0.1
        self.high = kwargs['high'] if 'high' in kwargs else 0.1
        self.initialize_parameters(self.loc, self.scale, self.low, self.high)

    #@jit(cache=True)
    def initialize_parameters(self, loc, scale, low, high):
        assert len(self.shape) >= 1, "shape must be a tuple of at least one element."

        if self.initialization_method=="normal":
            self.param = np.random.normal(loc=loc, scale=scale, size=self.shape)
        if self.initialization_method=="uniform":
            self.param = np.random.uniform(low=low, high=high, size=self.shape)
        elif self.initialization_method=="zero":
            self.param = np.zeros(shape=self.shape)
        elif self.initialization_method=="one":
            self.param = np.ones(shape=self.shape)
        elif self.initialization_method=='glorot':    
            a = np.sqrt(6.0 / np.sum(self.shape))
            self.param = np.random.uniform(-a, a, self.shape)

        self.dparam = np.zeros_like(self.param)
        self.dparam_1 = None

class NN(ABC):
    def __init__(self):
        self.params: dict[str, ArrayParam] = {}
        self.forward_dict: dict[str, type] = {}
        self.params = {}

    @abstractmethod
    def forward(self,):
        pass
    
    @abstractmethod
    def backward(self,):
        pass

    def __str__(self,):
        return self.__class__.__name__

    #@jit(cache=True)
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

class Linear(NN):
    ##@personal_log
    def __init__(self, dim_in, dim_out, weights_initialization_method='normal', bias: bool=True, bias_initialization_method='zeros'):
        super().__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.bias = bias
        self.weights_initialization_method = weights_initialization_method
        self.bias_initialization_method = bias_initialization_method

        self.params : dict[str, ArrayParam] = {}
        self.params['w'] = ArrayParam(shape=(dim_in, dim_out), initialization_method=weights_initialization_method)
        if self.bias: self.params['b'] = ArrayParam(shape=(1, dim_out), initialization_method=bias_initialization_method)

    ##@personal_log
    #@jit(cache=True)
    def forward(self, x, no_grad=False):
        if self.bias: a = x @ self.params['w'].param + self.params['b'].param
        else: a = x @ self.params['w'].param
        if not no_grad: 
            self.forward_dict = {
                'x': x,
                'a': a,
            }
        else:
            self.forward_dict = {
                'x': np.zeros_like(x),
                'a': np.zeros_like(a),
            }
        return a
    
    #@personal_log
    #@jit(cache=True)
    def backward(self, da):
        dw = self.forward_dict['x'].T @ da
        dx = da @ self.params['w'].param.T

        self.params['w'].dparam = dw
        #print(f"norm of weight of linear is {norm(dw):.4f}")

        if self.bias: 
            db = np.sum(da, axis=0)[np.newaxis, :]
            self.params['b'].dparam = db
        return dx
    
class Criterion(ABC):
    def __init__(self):
        self.forward_dict: dict[str, type] = {}
        self.params : dict[str, Type(Param)] = {}

    @abstractmethod
    def forward(self,):
        pass
    
    @abstractmethod
    def backward(self,):
        pass

    def __str__(self,):
        return self.__class__.__name__

    #@jit(cache=True)
    def __call__(self, x, y, no_grad=False):
        return self.forward(x, y, no_grad)
    
class CrossEntropy(Criterion):
    #@personal_log
    def __init__(self, epsilon=1e-20,):
        super().__init__()
        self.epsilon = epsilon

    #@jit(cache=True)
    def softmax(self, x):
        f = x - np.max(x, axis=1)[:,np.newaxis]
        p = np.exp(f) / np.sum(np.exp(f), axis=1)[:,np.newaxis]
        return p

    #@personal_log
    #@jit(cache=True)
    def forward(self, x, y, no_grad=False):
        m = y.shape[0]
        p = self.softmax(x)
        p[np.where(p < self.epsilon)] = self.epsilon
        p[np.where(p > 1 - self.epsilon)] = 1 - self.epsilon
        loss = - np.sum(y * np.log(p)) / m
        if not no_grad: 
            self.forward_dict = {
                'x': x,
                'y': y,
                'p': p,
                'loss': loss,
            }
        return p, loss
    
    #@personal_log
    #@jit(cache=True)
    def backward(self, out):
        m = self.forward_dict['y'].shape[0]
        grad = out - self.forward_dict['y']
        grad = grad/m # to take the mean across the batch
        return grad

In [881]:
class Activation():
    @staticmethod
    def relu(x, grad=False):
        if grad:
            return (x > 0) * 1
        else:
            return np.maximum(0, x)
        
    @staticmethod
    def sigmoid(x, grad=False):
        if grad:
            return (1 / (1 + np.exp(-x)))*(1 - (1 / (1 + np.exp(-x))))
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def tanh(x, grad=False):
        if grad:
            return 1 - np.tanh(x)**2
        return np.tanh(x)      
    
class Optimizer():
    def __init__(self, ):
        pass

    def __str__(self,):
        return self.__class__.__name__

    @abstractmethod
    def step(self, ):
        pass

    #@jit(cache=True)
    #@personal_log
    def zero_grad(self, ):
        # if to be rewritten, redirect to a function directly inside the class
        for layer in self.model.layers:
            for param in layer.params:
                layer.params[param].dparam = np.zeros_like(layer.params[param].param)

class SGDOptimizer(Optimizer):
    #@personal_log
    def __init__(self, model, lr, mu_momentum=0., tau_dampening=0., nesterov=False, weight_decay=0.,):
        super().__init__()
        self.lr = lr
        self.mu_momentum = mu_momentum
        self.model = model
        self.tau_dampening = tau_dampening
        self.nesterov = nesterov
        self.weight_decay = weight_decay

    #@personal_log
    #@jit(cache=True)
    def step(self, ):
        for layer in self.model.layers:
            for param in layer.params:
                if layer.params[param].is_trainable:
                    grad_t = layer.params[param].dparam
                    if self.weight_decay != 0.:
                        grad_t += self.weight_decay * layer.params[param].param
                    if self.mu_momentum != 0.:
                        bt_1 = layer.params[param].dparam_1
                        if bt_1 is not None:
                            bt = self.mu_momentum * bt_1 + (1 - self.tau_dampening) * grad_t
                        else:
                            bt = grad_t

                        if self.nesterov:
                            grad_t += self.mu_momentum * bt
                        else:
                            grad_t = bt

                    layer.params[param].param -= grad_t * self.lr

                    # post update
                    layer.params[param].dparam_1 = layer.params[param].dparam

class AdamOptimizer(Optimizer):
    def __init__(self, model, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        super().__init__()
        self.model = model
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.t = 1.

        self.params = self.get_model_parameters(self.model.layers)
        self.m_dw, self.v_dw = self.init_moments()

    def get_model_parameters(self, layers):
        if isinstance(layers, list):
            #print(f"list {layers}")
            elt = layers.pop(0)
            if len(layers) > 0:
                return self.get_model_parameters(elt) + self.get_model_parameters(layers)
            else:
                return self.get_model_parameters(elt)
        elif isinstance(layers, NN):
            #print(f"NN {layers}")
            if layers.params:
                return self.get_model_parameters(layers.params)
            elif layers.layers:
                return self.get_model_parameters(layers.layers)
            else:
                raise Exception('error type 2 ', layers.__class__.__name__)
        elif isinstance(layers, Criterion):
            return []
        elif isinstance(layers, dict):
            #print(f"dict {layers}")
            return list(layers.values())
        else:
            raise Exception('error type 1 ', layers.__class__.__name__)

    def init_moments(self, ):
        m_dw, v_dw = [], []

        for param in self.params:
            m_dw.append(np.zeros_like(param))
            v_dw.append(np.zeros_like(param))

        return m_dw, v_dw

    def step(self, ):
        for i, param in enumerate(self.params):
            if param.is_trainable:
                ## momentum beta 1
                # *** weights *** #
                self.m_dw[i] = self.beta1 * self.m_dw[i] + (1-self.beta1) * param.dparam

                ## rms beta 2
                # *** weights *** #
                self.v_dw[i] = self.beta2 * self.v_dw[i] + (1-self.beta2) * (param.dparam**2)

                ## bias correction
                m_dw_corr = self.m_dw[i]/(1-self.beta1**self.t)
                v_dw_corr = self.v_dw[i]/(1-self.beta2**self.t)

                ## update weights and biases
                param.param -=  self.lr * (m_dw_corr/(np.sqrt(v_dw_corr)+self.epsilon))
        self.t += 1

    def zero_grad(self, ):
        # if to be rewritten, redirect to a function directly inside the class
        for param in self.params:
            if param.is_trainable:
                param.dparam = np.zeros_like(param.param)

    def gradient_array(self, ):
        return np.concatenate([param.dparam.ravel() for param in self.params])

    def clip_gradient(self, max_threshold):
        gradient_arr = self.gradient_array()
        norm_ = np.linalg.norm(gradient_arr)
        if norm_ >= max_threshold:
            for param in self.params:
                param.dparam = max_threshold * param.dparam / norm_

    def clip_gradient_v2(self, max_threshold):
        for param in self.params:
            norm_ = np.linalg.norm(param.dparam)
            if norm_ >= max_threshold:
                param.dparam = max_threshold * param.dparam / norm_

In [829]:

def one_hot(x, dim):
    return np.eye(dim)[x]

class Embedding(NN):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.dim_in = dim_in
        self.dim_out = dim_out

        self.params : dict[str, ArrayParam] = {}
        self.params['w'] = ArrayParam(shape=(dim_in, dim_out), initialization_method='uniform', low=-0.1, high=0.1)

    def forward(self, x, no_grad=False):
        cx = x @ self.params['w'].param
        if not no_grad:
            self.forward_dict = {
                'x': x,
                'cx': cx,
            }
        return cx

    def backward(self, dcx):
        dw = self.forward_dict['x'].T @ dcx
        self.params['w'].dparam = dw
        #print(f"norm of weight of embedding is {norm(dw):.4f}")
        return dcx

# RNN

In [831]:
class RNN(NN):
    def __init__(self, dim_in, dim_hidden, nonlinearity=Activation.tanh, bias=True):
        super().__init__()
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        #self.num_layers = num_layers
        self.nonlinearity = nonlinearity
        self.bias = bias
        self.params: dict[str, ArrayParam] = {}
        b = np.sqrt(1 / self.dim_hidden)
        #self.params['w'] = ArrayParam(shape=(dim_in, dim_hidden), initialization_method='uniform', low=-b, high=b)
        #self.params['u'] = ArrayParam(shape=(dim_hidden, dim_hidden), initialization_method='uniform', low=-b, high=b)
        self.params['w'] = ArrayParam(shape=(dim_in + dim_hidden, dim_hidden), initialization_method='uniform', low=-b, high=b)
        if self.bias:
            self.params['b'] = ArrayParam(shape=(1, dim_hidden), initialization_method='uniform', low=-b, high=b)

    #def forward(self, cx, h_1, no_grad=False):
    def forward(self, combined, no_grad=False):
        #a = cx @ self.params['w'].param + h_1 @ self.params['u'].param
        a = combined @ self.params['w'].param
        if self.bias:
            a += self.params['b'].param
        h = self.nonlinearity(a, grad=False)
        if not no_grad:
            self.forward_dict = {
                'combined': combined,
                'a': a,
                'h': h,
            }
        return h

    #def backward(self, dltdh, dldh):
    def backward(self, dltdh, dldh):
        dldh += dltdh
        da = (1 - self.forward_dict['h'] ** 2) * dldh
        #dcx = da @ self.params['w'].param.T
        #dldh_1 = da @ self.params['u'].param.T
        dcombined = da @ self.params['w'].param.T

        #self.params['w'].dparam += self.forward_dict['cx'].T @ da
        #self.params['u'].dparam += self.forward_dict['h_1'].T @ da
        self.params['w'].dparam += self.forward_dict['combined'].T @ da
        if self.bias:
            self.params['b'].dparam += np.sum(da, axis=0)[np.newaxis, :]
        #return dldh_1, dcx
        dcx, dldh_1 = dcombined[:,:self.dim_in], dcombined[:,self.dim_in:] 
        #print(f"dcx shape is {dcx.shape}, and dldh_1 shape {dldh_1.shape}")
        return dcx, dldh_1

class RNNModel():
    def __init__(self, vocab_size, context_size=16, embed_size=128, hidden_size=256, bias=True):
        self.vocab_size = vocab_size
        self.bias = bias
        self.context_size = context_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.forward_dicts = []

        self.embedding: Embedding = Embedding(vocab_size, self.embed_size)
        self.rnn: RNN = RNN(self.embed_size, self.hidden_size, bias=self.bias)
        self.linear: Linear = Linear(self.hidden_size, self.vocab_size, bias=self.bias, weights_initialization_method='uniform', bias_initialization_method='uniform')
        self.cross_entropy: CrossEntropy = CrossEntropy()

        self.layers = [
            self.embedding,
            self.rnn,
            self.linear,
            self.cross_entropy,
        ]

    def __call__(self, x, y, no_grad=False):
        return self.forward(x, y, no_grad)

    def forward(self, x, y, no_grad=False):
        T, B, _ = x.shape
        probas = np.zeros((T, B, self.vocab_size))
        hidden = np.zeros((B, self.hidden_size))
        losses = np.zeros((T,))
        for timestamp in range(self.context_size):
            cxt = self.embedding(x[timestamp], no_grad)
            #hidden = self.rnn(cxt, hidden, no_grad)
            #print(f"combined shape is {np.vstack((cxt, hidden)).shape}")
            hidden = self.rnn(np.hstack((cxt, hidden)), no_grad)
            logits = self.linear(hidden, no_grad)
            probas[timestamp], losses[timestamp] = self.cross_entropy(logits, y[timestamp], no_grad)
            if not no_grad:
                self.forward_dicts.append((self.embedding.forward_dict, 
                                        self.rnn.forward_dict, 
                                        self.linear.forward_dict, 
                                        self.cross_entropy.forward_dict))
        return probas, losses
    
    def backward(self, out):
        T, B, C = out.shape
        dldh = np.zeros(shape=(B, self.hidden_size))
        for timestamp in reversed(range(self.context_size)):
            self.embedding.forward_dict, self.rnn.forward_dict, self.linear.forward_dict, self.cross_entropy.forward_dict = self.forward_dicts.pop()
            out = self.cross_entropy.backward(out[timestamp])
            dltdh = self.linear.backward(out)
            dcxt, dldh = self.rnn.backward(dltdh, dldh)
            _ = self.embedding.backward(dcxt)

In [832]:
# loading and preparing data
train_data, valid_data, test_data, word_to_id, id_2_word = ptb_raw_data(data_path=r'C:\Users\zarou\Documents\GitHub_Projects\RNN_LSTM_from_scratch_numpy\data')

In [827]:
batch_size = 128
context_size = 4

In [882]:
model = RNNModel(vocab_size=len(word_to_id), context_size=context_size, embed_size=128, hidden_size=128, bias=True) # 10001 token
#optimizer = SGDOptimizer(model=model, lr=0.01, mu_momentum=0., nesterov=False, weight_decay=0.)
optimizer = AdamOptimizer(model=model, lr=0.001)

In [883]:
# training
for step, (x, y) in enumerate(ptb_iterator(train_data, batch_size, context_size)):
    inputs = np.transpose(one_hot(x, len(word_to_id)), axes=(1,0,2)) # (B, T, C) transposed to (T, B, C)
    targets = np.transpose(one_hot(y, len(word_to_id)), axes=(1,0,2)) # (B, T, C) transposed to (T, B, C)
    
    optimizer.zero_grad()
    outputs, losses = model(inputs, targets, no_grad=False)
    model.backward(outputs)
    print(f"gradient before {np.linalg.norm(optimizer.params[0].dparam)}")
    optimizer.clip_gradient(0.25)
    print(f"gradient after {np.linalg.norm(optimizer.params[0].dparam)}")
    optimizer.step()

    if step % 1 == 0:
        print(f'step: {step} and loss: {np.mean(losses)}')

training data shape is 971657 and the number of batches is 7591
gradient before 0.04945506036477639
gradient after 0.029059771546244843
step: 0 and loss: 9.215868326213137
gradient before 0.044345489095827514
gradient after 0.027518467041726537
step: 1 and loss: 9.208911193524955
gradient before 0.04252131877984088
gradient after 0.029001373336545445
step: 2 and loss: 9.204193609989513
gradient before 0.04316562371465199
gradient after 0.02357117648278669
step: 3 and loss: 9.19696845843492
gradient before 0.042777915355527565
gradient after 0.023188137742257014
step: 4 and loss: 9.188605745355412
gradient before 0.05075395206377643
gradient after 0.025769916424339727
step: 5 and loss: 9.177690474968044
gradient before 0.04908700669741967
gradient after 0.02341178748537361
step: 6 and loss: 9.167860273058961
gradient before 0.04347566469133623
gradient after 0.022959583633121096
step: 7 and loss: 9.17369121849586
gradient before 0.04484555305186524
gradient after 0.019500911028370052
st

KeyboardInterrupt: 

# LSTM

In [905]:
class Gate(NN):
    def __init__(self, dim_in, dim_hidden, nonlinearity=Activation.sigmoid, bias=True):
        super().__init__()
        self.dim_in, self.dim_hidden, self.nonlinearity, self.bias = dim_in, dim_hidden, nonlinearity, bias
        self.params: dict[str, ArrayParam] = {}
        b = np.sqrt(1 / self.dim_hidden)
        self.params['w'] = ArrayParam(shape=(dim_in, dim_hidden), initialization_method='uniform', low=-b, high=b)
        self.params['u'] = ArrayParam(shape=(dim_hidden, dim_hidden), initialization_method='uniform', low=-b, high=b)
        if self.bias:
            self.params['b'] = ArrayParam(shape=(1, dim_hidden), initialization_method='uniform', low=-b, high=b)

    def forward(self, cx, h_1, no_grad=False):
        a = cx @ self.params['w'].param + h_1 @ self.params['w'].param
        if self.bias:
            a += self.params['b'].param
        h = self.nonlinearity(a, grad=False)
        if not no_grad:
            self.forward_dict = {
                'cx': cx,
                'h_1': h_1,
                'a': a,
                'h': h,
            }
        return h

    def backward(self, dh):
        da = self.nonlinearity(self.forward_dict['h'], grad=True) * dh
        dcx = da @ self.params['w'].param.T
        dh_1 = da @ self.params['u'].param.T
        self.params['w'].dparam += self.forward_dict['cx'].T @ da
        self.params['u'].dparam += self.forward_dict['h_1'].T @ da
        if self.bias:
            self.params['b'].dparam += np.sum(da, axis=0)[np.newaxis, :]
        return dcx, dh_1
        

class LSTM(NN):
    def __init__(self, dim_in, dim_hidden, bias=True):
        super().__init__()
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        #self.num_layers = num_layers
        self.bias = bias
        self.params: dict[str, ArrayParam] = {}
        # input, forget, output gates
        self.input_gate = Gate(self.dim_in, self.dim_hidden, nonlinearity=Activation.sigmoid, bias=self.bias)
        self.forget_gate = Gate(self.dim_in, self.dim_hidden, nonlinearity=Activation.sigmoid, bias=self.bias)
        self.output_gate = Gate(self.dim_in, self.dim_hidden, nonlinearity=Activation.sigmoid, bias=self.bias)
        # cell state
        self.cell_state = Gate(self.dim_in, self.dim_hidden, nonlinearity=Activation.tanh, bias=self.bias)

        self.layers = [
            self.input_gate,
            self.forget_gate,
            self.output_gate,
            self.cell_state
        ]

    def forward(self, cx, ht_1, ct_1, no_grad=False):
        it = self.input_gate(cx, ht_1)
        ft = self.forget_gate(cx, ht_1)
        ot = self.output_gate(cx, ht_1)
        ct_tilde = self.cell_state(cx, ht_1)
        ct = ft * ct_1 + it * ct_tilde
        ht = ot * Activation.tanh(ct)

        if not no_grad:
            self.forward_dict = {
                'cx': cx,
                'ht_1': ht_1,
                'ct_1': ct_1,
                'it': it,
                'ft': ft,
                'ot': ot,
                'ct_tilde': ct_tilde,
                'ct': ct,
                'ht': ht
            }

        return ht, ct

    def backward(self, dht, dh_, dc_):
        dh = dh_ + dht

        dc = dc_ + dh * self.forward_dict['ot'] * (1 - self.forward_dict['ct']**2)
        do = dh * Activation.tanh(self.forward_dict['ct'])
        
        di = dc * self.forward_dict['ct_tilde']
        df = dc * self.forward_dict['ct_1']
        dc_tilde = dc * self.forward_dict['it']

        dcx_i, dh_1_i = self.input_gate.backward(di)
        dcx_f, dh_1_f = self.forget_gate.backward(df)
        dcx_o, dh_1_o = self.output_gate.backward(do)
        dcx_c_tilde, dh_1_c_tilde = self.cell_state.backward(dc_tilde)

        dcx = dcx_i + dcx_f + dcx_o + dcx_c_tilde
        dh_1 = dh_1_i + dh_1_f + dh_1_o + dh_1_c_tilde
        dc_1 = dc * self.forward_dict['ft']

        return dcx, dh_1, dc_1

In [906]:
class LSTMModel():
    def __init__(self, vocab_size, context_size=16, embed_size=128, hidden_size=256, bias=True):
        self.vocab_size = vocab_size
        self.bias = bias
        self.context_size = context_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.forward_dicts = []

        self.embedding: Embedding = Embedding(vocab_size, self.embed_size)
        self.rnn: LSTM = LSTM(self.embed_size, self.hidden_size, bias=self.bias)
        self.linear: Linear = Linear(self.hidden_size, self.vocab_size, bias=self.bias, weights_initialization_method='uniform', bias_initialization_method='uniform')
        self.cross_entropy: CrossEntropy = CrossEntropy()

        self.layers = [
            self.embedding,
            self.rnn,
            self.linear,
            self.cross_entropy,
        ]
        
        self.layers.extend(self.rnn.layers)

    def __call__(self, x, y, no_grad=False):
        return self.forward(x, y, no_grad)

    def forward(self, x, y, no_grad=False):
        T, B, _ = x.shape
        probas = np.zeros((T, B, self.vocab_size))
        hidden = np.zeros((B, self.hidden_size))
        cell_state = np.ones((B, self.hidden_size))
        losses = np.zeros((T,))
        for timestamp in range(self.context_size):
            cxt = self.embedding(x[timestamp], no_grad)
            hidden, cell_state = self.rnn(cxt, hidden, cell_state, no_grad)
            logits = self.linear(hidden, no_grad)
            probas[timestamp], losses[timestamp] = self.cross_entropy(logits, y[timestamp], no_grad)
            if not no_grad:
                self.forward_dicts.append((self.embedding.forward_dict, 
                                        self.rnn.forward_dict, 
                                        self.linear.forward_dict, 
                                        self.cross_entropy.forward_dict,
                                        self.rnn.input_gate.forward_dict, 
                                        self.rnn.output_gate.forward_dict, 
                                        self.rnn.forget_gate.forward_dict, 
                                        self.rnn.cell_state.forward_dict))                
        return probas, losses
    
    def backward(self, out):
        T, B, C = out.shape
        dldh = np.zeros(shape=(B, self.hidden_size))
        dldc = np.ones(shape=(B, self.hidden_size))
        for timestamp in reversed(range(self.context_size)):
            self.embedding.forward_dict, self.rnn.forward_dict, self.linear.forward_dict, self.cross_entropy.forward_dict, self.rnn.input_gate.forward_dict, self.rnn.output_gate.forward_dict, self.rnn.forget_gate.forward_dict, self.rnn.cell_state.forward_dict = self.forward_dicts.pop()
            out = self.cross_entropy.backward(out[timestamp])
            dltdh = self.linear.backward(out)
            dcxt, dldh, dldc = self.rnn.backward(dltdh, dldh, dldc)
            self.embedding.backward(dcxt)

In [907]:
batch_size = 128
context_size = 4
hidden_size = 128
emb_size = 128

In [910]:
model = LSTMModel(vocab_size=len(word_to_id), context_size=context_size, embed_size=emb_size, hidden_size=hidden_size, bias=True) # 10001 token
#optimizer = SGDOptimizer(model=model, lr=0.01, mu_momentum=0., nesterov=False, weight_decay=0.)
optimizer = AdamOptimizer(model=model, lr=0.001)

In [911]:
# training
for step, (x, y) in enumerate(ptb_iterator(train_data, batch_size, context_size)):
    inputs = np.transpose(one_hot(x, len(word_to_id)), axes=(1,0,2)) # (B, T, C) transposed to (T, B, C)
    targets = np.transpose(one_hot(y, len(word_to_id)), axes=(1,0,2)) # (B, T, C) transposed to (T, B, C)
    
    optimizer.zero_grad()
    outputs, losses = model(inputs, targets, no_grad=False)
    model.backward(outputs)
    #optimizer.clip_gradient_v2(0.25)
    optimizer.step()

    if step % 1 == 0:
        print(f'step: {step} and loss: {np.mean(losses)}')

training data shape is 971657 and the number of batches is 7591
step: 0 and loss: 9.21472377620253
step: 1 and loss: 9.208540134477602
step: 2 and loss: 9.197862771020402
step: 3 and loss: 9.202880478227758
step: 4 and loss: 9.20105376981453
step: 5 and loss: 9.18967482118309
step: 6 and loss: 9.180327805947195
step: 7 and loss: 9.171154616186618
step: 8 and loss: 9.175932519362952
step: 9 and loss: 9.164491626246624
step: 10 and loss: 9.172525164460431
step: 11 and loss: 9.162257772634003
step: 12 and loss: 9.161896338318206
step: 13 and loss: 9.159668820408685
step: 14 and loss: 9.158038491156312
step: 15 and loss: 9.143932184044226
step: 16 and loss: 9.149290526476934
step: 17 and loss: 9.13794304916937
step: 18 and loss: 9.142540622149617
step: 19 and loss: 9.136955625723951
step: 20 and loss: 9.142086227552385
step: 21 and loss: 9.124938675482193
step: 22 and loss: 9.127532237644456
step: 23 and loss: 9.111840898149081
step: 24 and loss: 9.120658956856348
step: 25 and loss: 9.1039

KeyboardInterrupt: 

# Comparison with pytorch RNN

In [798]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_size, seq_len, hidden_size, batch_size, bias=True):
        super(RNN, self).__init__()
        self.vocab_size, self.emb_size, self.seq_len, self.hidden_size, self.batch_size, self.bias = vocab_size, emb_size, seq_len, hidden_size, batch_size, bias
        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.hidden = nn.Linear(emb_size + hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, vocab_size)
        self.init_weights()

    def init_weights(self):
        """Initialize the embedding and output weights uniformly."""
        # Intialize embedding weights unformly in the range [-0.1, 0.1]
        nn.init.uniform_(self.embeddings.weight, -0.1, 0.1)

        # Initialize the weights and biases uniformly
        b = 1/math.sqrt(self.hidden_size)
        nn.init.uniform_(self.hidden.weight, -b, b)
        nn.init.uniform_(self.hidden.bias, -b, b)

        # Initialize output layer weights uniformly in the range [-0.1, 0.1]
        # And all the biases to 0
        nn.init.uniform_(self.output.weight, -0.1, 0.1)
        nn.init.zeros_(self.output.bias)

    def init_hidden(self):
        """Initialize the hidden states to zero.

        This is used for the first mini-batch in an epoch, only.
        """
        return torch.zeros(self.batch_size, self.hidden_size)
    
    def forward(self, x, hidden_state):
        if x.is_cuda:
            device = x.get_device()
        else:
            device = torch.device("cpu")
        # Create a tensor to store outputs during the Forward
        outputs = torch.zeros(self.seq_len, self.batch_size, self.vocab_size).to(device)
        for timestamp in range(self.seq_len):
            cx = self.embeddings(x[timestamp])
            combined = torch.cat((cx, hidden_state), 1)
            pre_hidden = self.hidden(combined)
            hidden = torch.tanh(pre_hidden)
            outputs[timestamp] = self.output(hidden)
        return outputs, hidden

In [799]:
# Use the GPU if you have one
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device("cuda")
else:
    print("WARNING: You are about to run on cpu, and this will likely run out \
      of memory. \n You can try setting batch_size=1 to reduce memory usage")
    device = torch.device("cpu")

Using the GPU


In [800]:
emb_size=128
hidden_size=128
seq_len=4
batch_size=128
vocab_size=10001
#num_layers=1
#dp_keep_prob=1
initial_lr = 0.01
debug = True
num_epochs = 5
#save_best = False
#save_dir = "/"
#torch.autograd.set_detect_anomaly(False)

In [801]:
model = RNN(emb_size=emb_size, hidden_size=hidden_size,
            seq_len=seq_len, batch_size=batch_size,
            vocab_size=vocab_size)

model = model.to(device)

# LOSS FUNCTION
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr)

# LEARNING RATE
lr = initial_lr

In [802]:
def repackage_hidden(h):
    """
    Wraps hidden states in new Tensors, to detach them from their history.

    This prevents Pytorch from trying to backpropagate into previous input
    sequences when we use the final hidden states from one mini-batch as the
    initial hidden states for the next mini-batch.

    Using the final hidden states in this way makes sense when the elements of
    the mini-batches are actually successive subsequences in a set of longer sequences.
    This is the case with the way we've processed the Penn Treebank dataset.
    """
    if isinstance(h, Variable):
        return h.detach_()
    else:
        return tuple(repackage_hidden(v) for v in h)

def run_epoch(model, data, is_train=False, lr=1.0):
    """
    One epoch of training/validation (depending on flag is_train).
    """
    if is_train:
        model.train()
    else:
        model.eval()
    epoch_size = ((len(data) // model.batch_size) - 1) // model.seq_len
    start_time = time.time()
    hidden = model.init_hidden()
    hidden = hidden.to(device)

    costs = 0.0
    iters = 0
    losses = []

    # LOOP THROUGH MINIBATCHES
    for step, (x, y) in enumerate(ptb_iterator(data, model.batch_size, model.seq_len)):
        inputs = torch.from_numpy(x.astype(np.int64)).transpose(0, 1).contiguous().to(device)
        model.zero_grad()
        hidden = repackage_hidden(hidden)
        outputs, hidden = model(inputs, hidden)

        targets = torch.from_numpy(y.astype(np.int64)).transpose(0, 1).contiguous().to(device)
        tt = torch.squeeze(targets.view(-1, model.batch_size * model.seq_len))

        # LOSS COMPUTATION
        # This line currently averages across all the sequences in a mini-batch
        # and all time-steps of the sequences.
        # For problem 4.1, you will (instead) need to compute the average loss
        # at each time-step separately. Hint: use the method retain_grad to keep
        # gradients for intermediate nodes of the computational graph.
        #
        loss = loss_fn(outputs.contiguous().view(-1, model.vocab_size), tt)
        costs += loss.data.item() * model.seq_len
        losses.append(costs)
        iters += model.seq_len
        if debug:
            print(step, loss.item())
        if is_train:  # Only update parameters if training
            loss.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            optimizer.step()
            if step % (epoch_size // 10) == 10:
                print('step: '+ str(step) + '\t' \
                    + "loss (sum over all examples' seen this epoch):" + str(costs) + '\t' \
                    + 'speed (wps):' + str(iters * model.batch_size / (time.time() - start_time)))
    return np.exp(costs / iters), losses

In [803]:
print("\n########## Running Main Loop ##########################")
train_ppls = []
train_losses = []
val_ppls = []
val_losses = []
best_val_so_far = np.inf
times = []

# In debug mode, only run one epoch
if debug:
    num_epochs = 1
else:
    num_epochs = num_epochs

# MAIN LOOP
for epoch in range(num_epochs):
    t0 = time.time()
    print('\nEPOCH '+str(epoch)+' ------------------')
    
    # RUN MODEL ON TRAINING DATA
    train_ppl, train_loss = run_epoch(model, train_data, True, lr)

    # RUN MODEL ON VALIDATION DATA
    val_ppl, val_loss = run_epoch(model, valid_data)


    # SAVE MODEL IF IT'S THE BEST SO FAR
    if val_ppl < best_val_so_far:
        best_val_so_far = val_ppl

    # LOC RESULTS
    train_ppls.append(train_ppl)
    val_ppls.append(val_ppl)
    train_losses.extend(train_loss)
    val_losses.extend(val_loss)
    times.append(time.time() - t0)
    log_str = 'epoch: ' + str(epoch) + '\t' \
            + 'train ppl: ' + str(train_ppl) + '\t' \
            + 'val ppl: ' + str(val_ppl)  + '\t' \
            + 'best val: ' + str(best_val_so_far) + '\t' \
            + 'time (s) spent in epoch: ' + str(times[-1])
    print(log_str)


########## Running Main Loop ##########################

EPOCH 0 ------------------
training data shape is 971657 and the number of batches is 7591
0 9.213391304016113
1 9.215801239013672
2 9.21054458618164
3 9.215188980102539
4 9.216226577758789
5 9.214129447937012
6 9.213945388793945
7 9.212898254394531
8 9.213916778564453
9 9.207658767700195
10 9.212010383605957
step: 10	loss (sum over all examples' seen this epoch):405.3828468322754	speed (wps):6642.81453548031
11 9.206123352050781
12 9.211172103881836
13 9.213326454162598
14 9.209976196289062
15 9.20836353302002
16 9.211698532104492
17 9.207874298095703
18 9.20516300201416
19 9.207450866699219
20 9.212579727172852
21 9.205589294433594
22 9.205440521240234
23 9.203186988830566
24 9.205988883972168
25 9.209946632385254
26 9.207103729248047
27 9.207426071166992
28 9.206121444702148
29 9.20392894744873
30 9.207456588745117
31 9.20374584197998
32 9.204540252685547
33 9.209150314331055
34 9.203968048095703
35 9.203532218933105
36 9.202

KeyboardInterrupt: 

# Pytorch LSTM

In [889]:
# Use the GPU if you have one
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device("cuda")
else:
    print("WARNING: You are about to run on cpu, and this will likely run out \
      of memory. \n You can try setting batch_size=1 to reduce memory usage")
    device = torch.device("cpu")

Using the GPU


In [890]:
def repackage(h):
    """
    Wraps hidden states in new Tensors, to detach them from their history.

    This prevents Pytorch from trying to backpropagate into previous input
    sequences when we use the final hidden states from one mini-batch as the
    initial hidden states for the next mini-batch.

    Using the final hidden states in this way makes sense when the elements of
    the mini-batches are actually successive subsequences in a set of longer sequences.
    This is the case with the way we've processed the Penn Treebank dataset.
    """
    if isinstance(h, Variable):
        return h.detach_()
    else:
        return tuple(repackage(v) for v in h)

class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, seq_len, hidden_size, batch_size, bias=True):
        super(LSTM, self).__init__()
        self.vocab_size, self.emb_size, self.seq_len, self.hidden_size, self.batch_size, self.bias = vocab_size, emb_size, seq_len, hidden_size, batch_size, bias
        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.LSTM(emb_size, hidden_size)
        self.output = nn.Linear(hidden_size, vocab_size)

    def init_hidden_and_cell(self):
        """Initialize the hidden states to zero.

        This is used for the first mini-batch in an epoch, only.
        """
        return torch.zeros(1, self.batch_size, self.hidden_size), torch.zeros(1, self.batch_size, self.hidden_size)
    
    def forward(self, x, h_1, c_1):
        x = x.to(device)
        cx = self.embeddings(x)
        output, (h, c) = self.rnn(cx, (h_1, c_1))
        outputs = self.output(output).to(device)
        return outputs, (h, c)

In [903]:
emb_size=128
hidden_size=128
seq_len=4
batch_size=128
vocab_size=10001
lr = 0.001
num_epochs = 5

model = LSTM(emb_size=emb_size, hidden_size=hidden_size,
            seq_len=seq_len, batch_size=batch_size,
            vocab_size=vocab_size)

model = model.to(device)

# LOSS FUNCTION
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [902]:
def train(model, data, n_epochs=5):
    """
    One epoch of training/validation (depending on flag is_train).
    """
    for epoch in range(n_epochs):
        print(f"####################### EPOCH {epoch} #########################")
        h, c = model.init_hidden_and_cell()
        h, c = h.to(device), c.to(device)

        # LOOP THROUGH MINIBATCHES
        for step, (x, y) in enumerate(ptb_iterator(data, model.batch_size, model.seq_len)):
            inputs = torch.from_numpy(x.astype(np.int64)).transpose(0, 1).contiguous().to(device)
            model.zero_grad()
            h, c = repackage(h), repackage(c)
            outputs, (h, c) = model(inputs, h, c)

            targets = torch.from_numpy(y.astype(np.int64)).transpose(0, 1).contiguous().to(device)
            tt = torch.squeeze(targets.view(-1, model.batch_size * model.seq_len))

            loss = loss_fn(outputs.contiguous().view(-1, model.vocab_size), tt)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            optimizer.step()

            print(f"step: {step} loss: {loss.item()}")

train(model, train_data, 5)

####################### EPOCH 0 #########################
training data shape is 971657 and the number of batches is 7591
step: 0 loss: 9.208250045776367
step: 1 loss: 9.178266525268555
step: 2 loss: 9.154075622558594
step: 3 loss: 9.094618797302246
step: 4 loss: 9.034873962402344
step: 5 loss: 8.936956405639648
step: 6 loss: 8.889568328857422
step: 7 loss: 8.86879825592041
step: 8 loss: 8.550055503845215
step: 9 loss: 8.076891899108887
step: 10 loss: 7.584944725036621
step: 11 loss: 7.0122857093811035
step: 12 loss: 7.219669342041016
step: 13 loss: 6.65168571472168
step: 14 loss: 6.986882209777832
step: 15 loss: 6.6848883628845215
step: 16 loss: 7.011778831481934
step: 17 loss: 6.733390808105469
step: 18 loss: 6.834959506988525
step: 19 loss: 6.762497425079346
step: 20 loss: 6.74285364151001
step: 21 loss: 6.373056411743164
step: 22 loss: 6.451379776000977
step: 23 loss: 6.67000150680542
step: 24 loss: 6.749824523925781
step: 25 loss: 6.369228363037109
step: 26 loss: 6.415034770965576

KeyboardInterrupt: 